In [1]:
import uvicorn
from fastapi import FastAPI
import numpy as np
import io

In [2]:
from transformers import DonutProcessor, VisionEncoderDecoderModel
import torch



In [8]:
model = VisionEncoderDecoderModel.from_pretrained("Edgar404/donut-shivi-cheques_KD_320" ,torch_dtype = torch.float16 )
processor = DonutProcessor.from_pretrained("Edgar404/donut-shivi-cheques_KD_320")
model.eval()

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


VisionEncoderDecoderModel(
  (encoder): DonutSwinModel(
    (embeddings): DonutSwinEmbeddings(
      (patch_embeddings): DonutSwinPatchEmbeddings(
        (projection): Conv2d(3, 128, kernel_size=(4, 4), stride=(4, 4))
      )
      (norm): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.0, inplace=False)
    )
    (encoder): DonutSwinEncoder(
      (layers): ModuleList(
        (0): DonutSwinStage(
          (blocks): ModuleList(
            (0-1): 2 x DonutSwinLayer(
              (layernorm_before): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
              (attention): DonutSwinAttention(
                (self): DonutSwinSelfAttention(
                  (query): Linear(in_features=128, out_features=128, bias=True)
                  (key): Linear(in_features=128, out_features=128, bias=True)
                  (value): Linear(in_features=128, out_features=128, bias=True)
                  (dropout): Dropout(p=0.0, inplace=False)
           

In [9]:
from PIL import Image
from fastapi import FastAPI, UploadFile, File, HTTPException
from pydantic import BaseModel

In [10]:
from utils import process_image

In [ ]:
img = Image.open('shivi.jpg')

In [ ]:
results = process_image(img ,model , processor , torch.float16)

In [9]:
results

{'payee_name': 'Thérèse Fortier',
 'cheque_date': '06/05/22',
 'bank_name': 'AXIS BANK',
 'amt_in_words': 'Four Thousand Nine Hundred and Seventy One',
 'amt_in_figures': '4971'}

In [11]:
# building the server
app = FastAPI(title = 'Invoice parser')

In [12]:
@app.get('/')
def read_root():
    return {"hello":"world"}

In [13]:
@app.on_event('startup')
async def load_model():
    model = VisionEncoderDecoderModel.from_pretrained("Edgar404/donut-shivi-cheques_KD_320" ,torch_dtype = torch.float16 )
    processor = DonutProcessor.from_pretrained("Edgar404/donut-shivi-cheques_KD_320")
    model.eval()
    

C:\Users\GLC\AppData\Local\Temp\ipykernel_13188\2606298389.py:1: DeprecationWarning: 
        on_event is deprecated, use lifespan event handlers instead.

        Read more about it in the
        [FastAPI docs for Lifespan Events](https://fastapi.tiangolo.com/advanced/events/).
        
  @app.on_event('startup')


In [14]:
class InferenceResults(BaseModel):
    prediction:dict
    

In [16]:
@app.post('/predict')
async def get_prediction(file: UploadFile = File(...)):
    
    # 1. VALIDATE INPUT FILE
    filename = file.filename
    fileExtension = filename.split(".")[-1] in ("jpg", "jpeg", "png")
    if not fileExtension:
        raise HTTPException(status_code=415, detail="Unsupported file provided.")
        
    # 2. TRANSFORM RAW IMAGE INTO PIL image
    
    # Read image as a stream of bytes
    image_stream = io.BytesIO(file.file.read())
    
    # Start the stream from the beginning (position zero)
    image_stream.seek(0)
    
    # Write the stream of bytes into a numpy array
    file_bytes = np.asarray(bytearray(image_stream.read()), dtype=np.uint8)
    
    #Decode the numpy array as a PIL  image
    image = Image.fromarray(file_bytes)
    image = image.convert('RGB')
    
    # make the prediction
    prediction = process_image(image ,model , processor , torch.float16)
    
    return InferenceResults(prediction = prediction)
    

In [17]:
import os
import nest_asyncio

In [ ]:
# Allows the server to be run in this interactive environment
nest_asyncio.apply()

# Host depends on the setup you selected (docker or virtual env)
host = "0.0.0.0" if os.getenv("DOCKER-SETUP") else "127.0.0.1"

# Spin up the server!    
uvicorn.run(app, host=host, port=8000)

INFO:     Started server process [13188]
INFO:     Waiting for application startup.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)
